<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>
ENTSO-E Data / Data download notebook</p><br>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>

Download the dataset
</p>

This notebook uses data from the ENTSO-E Transparency Platform.

The data is stored in a SFTP server. To access and retrieve the data, it is necessary to be registered on the Transparency Platform.

The notebook automatically downloads data without using an external SFTP client software, allowing the user to choose the type and time period of the data.

The notebook also allows to select and upload data as pandas' DataFrames.

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>

Set up
</p>

**Import libraries**

In [1]:
import pandas as pd
import numpy as np

#Module that provides a portable way of using operating system dependent functionality
import os

#Python implementation of the SSHv2 protocol, providing both client and server functionality
#http://www.paramiko.org/
import paramiko

#Module to interact with the Notebook environment 
#https://ipywidgets.readthedocs.io/en/stable/
import ipywidgets as widgets

#Module for character encoding auto-detection 
#https://chardet.readthedocs.io/en/latest/index.html
import chardet 

#Cross-platform colored terminal text 
#https://pypi.org/project/colorama/
from colorama import Fore
from colorama import Style

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>

User defined parameters
</p>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>

Defining access credentials
</p>

The credentials to access the SFTP server should be saved on a .txt file titled **"ENTSO-E-Credentials.txt"** located in your user profile path (usually something like C:\Users\youruser\) with the following format (**No blank spaces**):

username:your.email@email.com<br>
password:SecretPassword<br>
hostname:sftp-transparency.entsoe.eu<br>

**The .txt file and the Notebook should be saved on the same directory**

In [2]:
try:
    credentials_file_path = os.path.join(os.environ['USERPROFILE'], "ENTSO-E-Credentials.txt")
    print('Looking for your credentials in '+credentials_file_path)
    credentials = open(credentials_file_path, mode = 'r')
    for line in credentials:
        pieces = line.split(':')
        if pieces[0] == "username":
            #The rstrip() method returns a copy of the string with trailing characters removed (e.g. \n )
            username = pieces[1].rstrip()
        elif pieces[0] == 'password':
            password = pieces[1].rstrip()
        elif pieces[0] == 'hostname':
            hostname = pieces[1]
    
    print(Fore.BLUE + 'Welcome ' + username + '!')
    print(Style.RESET_ALL)
                   
except FileNotFoundError:
            print(Fore.RED + 'Please create a .txt file with your access credentials')
            print(Style.RESET_ALL)
            pass
        
except NameError:
            print(Fore.RED + 'Please check the format of your credentials in the .txt file. Remember, no blank spaces!')
            print(Style.RESET_ALL)
            pass
    

Looking for your credentials in C:\Users\alida\ENTSO-E-Credentials.txt
Welcome ali.darudi@unibas.ch!



**Directories**

In [3]:
#Paste here:
#The directory for downloaded data under "dir_source"
#The directory for parsed data under "dir_parsed"

#Note 1: If the directories are not given, they will be created automatically

#Note 2: The notation "./" indicates that the directory "/Data/source_data(parsed_data)" will be created in the 
#folder where the Notebook is saved

dir_source = "./Data/source_data"
dir_parsed = "./Data/parsed_data"

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>

Download functions
</p>

In [4]:
def filename(folder, year_start, month_start, year_end, month_end):
    """Returns a list with the filenames to be downloaded from the SFTP server
    :param folder: <str> The server's folder where the desired data is stored
    :param year_start: <str> Initial year of the required data's range
    :param month_start: <str> Initial month of the required data's range
    :param year_end: <str> Final year of the required data's range
    :param month_end: <str> Final month of the required data's range
    :return f_names: <list> List of the files' names"""
    year_endrg = year_end + 1
    f_names = []
    #looping through the selected time period
    for y in range(year_start, year_endrg):
        for m in range(1, 13):
            if month_start > m and y == year_start:
                pass
            else:   
                filename = str(y) + "_" + str(m) + "_" + folder + ".csv"
                f_names.append(filename)
                if m == month_end and y == year_end:
                    return f_names
                    break
    
    #Handling a nonchronological selection of dates
    print(Fore.RED + "Please choose the dates in chronological order")
    print(Style.RESET_ALL)
    return []

def create_directory(dir_selec):
    """Creates a new directory if one is not already created
    :param dir_selec: <str> path to desired directory
    :return dir_selec: <str> path to desired directory"""
    if not os.path.exists(dir_selec):
        os.makedirs(dir_selec)
    print("The directory is " + dir_selec)
    return dir_selec
    
def get_folders(hostname):
    """Returns a list of the folders in the SFTP server
    :param hostname: <str>  Hostname of the SFTP server
    :return: <list> list of folders"""
    
    try:
        #Connecting to the sftp server
        ssh_client = paramiko.SSHClient()

        #Automatically add keys without requiring human intervention
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        ssh_client.connect(hostname, username = username, password = password)

        sftp = ssh_client.open_sftp()
    
        folders = []
    
    except Exception:
        print(Fore.RED + 'Your credentials are not valid. Please check them or register in the ENTSO-E Transparency Platform')
        print(Style.RESET_ALL)
        return []
    
    try:
        #creates a list of the folders contained in the ftp server
        #folders = ftp.nlst()
        dir_sftp = "/TP_export"
        
        #Changing the current working directory (cwd) of the sftp server
        sftp.chdir(dir_sftp)
        
        #Retrieving the titles of all files within the directory of interest
        folders = sftp.listdir()
        
    except error_perm:
        if str(resp)== "500 No files found":
            print ("No files in this directory")
        else:
            raise
            
    sftp.close()
    ssh_client.close()
    
    return sorted(folders)
    
def download_file(hostname, cwd_url, username, password, dir_source):
    """Downloads the csv files from the selected folder on the ftp server
    :param hostname: <str> Hostname of the SFTP server
    :param cwd_url: <str> Directory containing the selected files (Current Working Directory)
    :param filename: <str> Name of the file to be retrieved
    :param username: <str> Username to accesss the ftp server
    :param password: <str> Password to access the ftp server
    :param dir_source: <str> directory path for the downloaded files
    return: None"""
    
    #A list of all files to be downloaded is created
    f_names = filename(folder, year_start, month_start, year_end, month_end)
    
    for fn in f_names:
        print(Fore.BLUE + "The filename is " + fn)
        print(Style.RESET_ALL)
        directory = str(create_directory(dir_source) + "/" + fn)
        sftp_down = str(cwd_url + "/" + fn)

        try:
            #Initializing a SSH client
            ssh_client = paramiko.SSHClient()
            #Automatically add keys without requiring human intervention
            ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            #Connecting to the SFTP server
            ssh_client.connect(hostname, username = username, password = password)
            sftp = ssh_client.open_sftp()
            
            #Retrieving the selected files
            sftp.get(sftp_down,directory)
            
        except FileNotFoundError:
            print(Fore.RED + 'The file ' + fn + ' does not exist')
            print(Style.RESET_ALL)
            pass
    
    try:
        #Disconnecting from the SFTP server
        sftp.close()
        ssh_client.close()
    except UnboundLocalError:
        pass

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>

Selection of download parameters
</p>

In [5]:
#https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html
folder_widget = widgets.Dropdown(
    options = get_folders(hostname),
    description = "Folder ",
    disabled = False)
display(folder_widget)

year_start_widget = widgets.Dropdown(
    options = ["2014","2015", "2016", "2017", "2019", "2020"],
    description = "Start year",
    disabled = False)
display(year_start_widget)

month_start_widget = widgets.Dropdown(
    options = ["1","2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"],
    description = "Start month",
    disabled = False)
display(month_start_widget)

year_end_widget = widgets.Dropdown(
    options = ["2014","2015", "2016", "2017", "2018", "2019", "2020"],
    description = "End year",
    disabled = False)
display(year_end_widget)

month_end_widget = widgets.Dropdown(
    options = ["1","2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"],
    description = "End month",
    disabled = False)
display(month_end_widget)

Dropdown(description='Folder ', options=('AcceptedAggregatedOffers', 'ActivatedBalancingEnergy', 'ActualGenera…

Dropdown(description='Start year', options=('2014', '2015', '2016', '2017', '2019', '2020'), value='2014')

Dropdown(description='Start month', options=('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'), v…

Dropdown(description='End year', options=('2014', '2015', '2016', '2017', '2018', '2019', '2020'), value='2014…

Dropdown(description='End month', options=('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'), val…

In [6]:
folder = folder_widget.value

year_start = int(year_start_widget.value)

month_start = int(month_start_widget.value)

year_end = int(year_end_widget.value)

month_end = int(month_end_widget.value)

**Defining the currect working directory (cwd)**

In [7]:
#cwd_url is the directory containing the selected files (Current Working Directory)
cwd_url = "/TP_export/" + folder + "/"
print("The cwd_url is " + cwd_url)
print("The folder is " + folder)

The cwd_url is /TP_export/CrossBorderPhysicalFlow/
The folder is CrossBorderPhysicalFlow


**Downloading files from the selected folder and range**

In [8]:
download_file(hostname, cwd_url, username, password, dir_source)

The filename is 2019_1_CrossBorderPhysicalFlow.csv

The directory is ./Data/source_data
The filename is 2019_2_CrossBorderPhysicalFlow.csv

The directory is ./Data/source_data
The filename is 2019_3_CrossBorderPhysicalFlow.csv

The directory is ./Data/source_data
The filename is 2019_4_CrossBorderPhysicalFlow.csv

The directory is ./Data/source_data
The filename is 2019_5_CrossBorderPhysicalFlow.csv

The directory is ./Data/source_data
The filename is 2019_6_CrossBorderPhysicalFlow.csv

The directory is ./Data/source_data
The filename is 2019_7_CrossBorderPhysicalFlow.csv

The directory is ./Data/source_data
The filename is 2019_8_CrossBorderPhysicalFlow.csv

The directory is ./Data/source_data
The filename is 2019_9_CrossBorderPhysicalFlow.csv

The directory is ./Data/source_data
The filename is 2019_10_CrossBorderPhysicalFlow.csv

The directory is ./Data/source_data
The filename is 2019_11_CrossBorderPhysicalFlow.csv

The directory is ./Data/source_data
The filename is 2019_12_CrossB

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>

Creating the DataFrames
</p>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>

Upload functions
</p>

In [9]:
def encoding(dir_source, fn):
    """Returns the encoding of the dowloaded files to allow its upload as DataFrames
    :param: dir_source: <str> directory path for the downloaded files
    return: code: <str> Encoding type"""

    code_path = dir_source + "/" + fn
    code_file = open(code_path, 'rb').read()
        
    #The "detect" function (chardet) function takes one argument, a non-Unicode string. 
    #It returns a dictionary containing the auto-detected character encoding
    result = chardet.detect(code_file)
        
    code = result['encoding']
    return code    
        
def DF_name(f_names, folder_up):
    """Returns a list with new names for the DataFrames
    :param f_names: <list> List containing the names of the filses to be retrieved
    :param folder: <str> The server's folder where the desired data is stored
    :returns: DF_names: <list>"""

    months = {
        '1': 'jan', '2': 'feb', '3': 'mar', '4':'apr', '5':'may', '6': 'jun',
    '7': 'jul', '8': 'aug', '9':'sep', '10': 'oct', '11': 'nov', '12': 'dec'}
    
    DF_names = []
            
    for fn in f_names:
        year = fn[0:4]
        if fn[6] == '_':
            month = months.get(fn[5])
        else:
            month = months.get(fn[5:7])
        name = month + year + folder_up
        DF_names.append(name)
                 
    return DF_names   
      
def DFfile_dir(dir_source):
    """Returns a dictionary of DataFrames from the selected files, which are retrieved from
    a directory on the computer
    :param: dir_source: <str> directory path for the downloaded files
    return: list_of_DFs: <dict> key: names of DataFrames, value: DataFrames of the selected data"""
    #A list of all files to be downloaded is created
    f_names = filename(folder_up, yearup_start, monthup_start, yearup_end, monthup_end)
    #print(f_names)
    #A list of the new names of the DataFrames is created
    DF_names = DF_name(f_names, folder_up)
    #print(DF_names)
    
    list_of_DFs = {}
    #List of columns to be dropped when creating the DataFrames. Write the columns you wish to delete from your DataFrame
    #Example: cols_drop = ['ActualConsumption', 'UpdateTime']
    cols_drop = []
            
    #The zip() function take iterables and returns an iterator of tuples.   
    for DF, fn in zip(DF_names, f_names):
        up_path = dir_source + "/" + fn
        print("The up path is " + up_path)

        try:
            list_of_DFs[DF] = pd.read_csv(up_path, sep = "\t", encoding = encoding(dir_source, fn)).drop(cols_drop, axis = 1)
            print(Fore.BLUE + "The name is " + DF)
            print(Style.RESET_ALL)          
                          
        except FileNotFoundError:
            print(Fore.RED + "The file " + fn + " is not saved in the selected directory")
            print(Style.RESET_ALL)
            pass
                
    return list_of_DFs


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>

Selection of upload parameters
</p>

**Widget List available in** https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html

In [10]:
folderup_widget = widgets.Dropdown(
    options = get_folders(hostname),
    description = "Folder ",
    disabled = False)
display(folderup_widget)

#folderup_widget = widgets.Dropdown(
    #options = ["AggregatedGenerationPerType"],
    #description = "Folder ",
    #disabled = False)
#display(folderup_widget)

yearup_start_widget = widgets.Dropdown(
    options = ["2014","2015", "2016", "2017", "2018", "2019", "2020"],
    description = "Start year",
    disabled = False)
display(yearup_start_widget)

monthup_start_widget = widgets.Dropdown(
    options = ["1","2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"],
    description = "Start month",
    disabled = False)
display(monthup_start_widget)

yearup_end_widget = widgets.Dropdown(
    options = ["2014","2015", "2016", "2017", "2018", "2019", "2020"],
    description = "End year",
    value="2019",
    disabled = False)
display(yearup_end_widget)

monthup_end_widget = widgets.Dropdown(
    options = ["1","2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"],
    value="12",
    description = "End month",
    disabled = False)
display(monthup_end_widget)

Dropdown(description='Folder ', options=('AcceptedAggregatedOffers', 'ActivatedBalancingEnergy', 'ActualGenera…

Dropdown(description='Start year', options=('2014', '2015', '2016', '2017', '2018', '2019', '2020'), value='20…

Dropdown(description='Start month', options=('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'), v…

Dropdown(description='End year', index=5, options=('2014', '2015', '2016', '2017', '2018', '2019', '2020'), va…

Dropdown(description='End month', index=11, options=('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', …

In [11]:
folder_up = folderup_widget.value

yearup_start = int(yearup_start_widget.value)

monthup_start = int(monthup_start_widget.value)

yearup_end = int(yearup_end_widget.value)

monthup_end = int(monthup_end_widget.value)

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>

Creating main dictionary of DataFrames
</p>

In [12]:
list_of_DFs = DFfile_dir(dir_source)

The up path is ./Data/source_data/2019_1_CrossBorderPhysicalFlow.csv
The name is jan2019CrossBorderPhysicalFlow

The up path is ./Data/source_data/2019_2_CrossBorderPhysicalFlow.csv
The name is feb2019CrossBorderPhysicalFlow

The up path is ./Data/source_data/2019_3_CrossBorderPhysicalFlow.csv
The name is mar2019CrossBorderPhysicalFlow

The up path is ./Data/source_data/2019_4_CrossBorderPhysicalFlow.csv
The name is apr2019CrossBorderPhysicalFlow

The up path is ./Data/source_data/2019_5_CrossBorderPhysicalFlow.csv
The name is may2019CrossBorderPhysicalFlow

The up path is ./Data/source_data/2019_6_CrossBorderPhysicalFlow.csv
The name is jun2019CrossBorderPhysicalFlow

The up path is ./Data/source_data/2019_7_CrossBorderPhysicalFlow.csv
The name is jul2019CrossBorderPhysicalFlow

The up path is ./Data/source_data/2019_8_CrossBorderPhysicalFlow.csv
The name is aug2019CrossBorderPhysicalFlow

The up path is ./Data/source_data/2019_9_CrossBorderPhysicalFlow.csv
The name is sep2019CrossBor

In [62]:
list_of_DFs

{'jan2019AggregatedGenerationPerType':          Year  Month  Day                 DateTime ResolutionCode  \
 0        2019      1    1  2019-01-01 00:00:00.000          PT60M   
 1        2019      1    1  2019-01-01 01:00:00.000          PT60M   
 2        2019      1    1  2019-01-01 02:00:00.000          PT60M   
 3        2019      1    1  2019-01-01 03:00:00.000          PT60M   
 4        2019      1    1  2019-01-01 04:00:00.000          PT60M   
 ...       ...    ...  ...                      ...            ...   
 1196488  2019      1   27  2019-01-27 19:00:00.000          PT60M   
 1196489  2019      1   27  2019-01-27 20:00:00.000          PT60M   
 1196490  2019      1   27  2019-01-27 21:00:00.000          PT60M   
 1196491  2019      1   27  2019-01-27 22:00:00.000          PT60M   
 1196492  2019      1   27  2019-01-27 23:00:00.000          PT60M   
 
                  areacode AreaTypeCode      AreaName MapCode ProductionType  \
 0        10YCH-SWISSGRIDZ          CTA 

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>

Exporting the DataFrames as csv files
</p>

# YOUR OWN SCRIPTING HERE

# Final writing of results (after your own scripting is done)

In [13]:
download_widget = widgets.RadioButtons(
    options=['Yes', 'No'],
    description='Save all files?',
    disabled=False)

display(download_widget)

RadioButtons(description='Save all files?', options=('Yes', 'No'), value='Yes')

In [14]:
download_csv = str(download_widget.value)
print(download_csv)

Yes


In [15]:
create_directory(dir_parsed)

#Saving the files in the selected path (dir_parsed folder)
if download_csv is 'Yes':
    for key, value in list_of_DFs.items():
        dir_csv = dir_parsed + "/" + key + ".csv"
        print(dir_csv)
        list_of_DFs[key].to_csv(dir_csv, sep = ',', index = False)

else:
    #Defining the DataFrame to plot
    save_list = []

    #Creating a list with the names of the available DataFrames
    for key in list_of_DFs.keys():
        save_list.append(key)

    save_widget = widgets.Dropdown(
        options = save_list,
        description = "File's name",
        disabled = False)
    display(save_widget)

The directory is ./Data/parsed_data
./Data/parsed_data/jan2019CrossBorderPhysicalFlow.csv
./Data/parsed_data/feb2019CrossBorderPhysicalFlow.csv
./Data/parsed_data/mar2019CrossBorderPhysicalFlow.csv
./Data/parsed_data/apr2019CrossBorderPhysicalFlow.csv
./Data/parsed_data/may2019CrossBorderPhysicalFlow.csv
./Data/parsed_data/jun2019CrossBorderPhysicalFlow.csv
./Data/parsed_data/jul2019CrossBorderPhysicalFlow.csv
./Data/parsed_data/aug2019CrossBorderPhysicalFlow.csv
./Data/parsed_data/sep2019CrossBorderPhysicalFlow.csv
./Data/parsed_data/oct2019CrossBorderPhysicalFlow.csv
./Data/parsed_data/nov2019CrossBorderPhysicalFlow.csv
./Data/parsed_data/dec2019CrossBorderPhysicalFlow.csv


In [46]:
if download_csv is 'No':
    save_file = str(save_widget.value)
    print(save_file)
    
    for key, value in list_of_DFs.items():
        dir_csv = dir_parsed + "/" + save_file + ".csv"
        if key == save_file:
            list_of_DFs[key].to_csv(dir_csv, sep = ',', index = False)
    
else:
    pass